<a href="https://colab.research.google.com/github/carloseducorinto/Agents/blob/main/Agents_Code_Generation_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Multi-Agent System for Python Code Generation in Data Analytics

In this example, we will build a multi-agent system to generate Python code for data analytics. This system will streamline the creation of data analytics scripts, making the process more efficient and automated.

Importing Required Libraries:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq
```

In [142]:
#!pip install -q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
#!pip install -q langchain_groq


In [143]:

import warnings
from crewai import Agent, Crew, Task
import os
from google.colab import userdata
from langchain_groq import ChatGroq
from crewai import Agent, Task, Crew
from crewai import Crew, Process
from langchain_openai import ChatOpenAI




# Warning control
warnings.filterwarnings('ignore')

In [144]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain.agents import Tool

#python_repl = PythonREPL()


# Create and assign the search tool to an agent
#python_repl_tool = Tool(
#  name="Python Tool",
#  func=python_repl.run,
#  description="Useful for running and debugin python code",
#)



In [145]:
#!pip install langchain_experimental

In [146]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
#os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
#llm=ChatGroq(temperature=0, model_name="llama3-70b-8192", api_key=userdata.get("GROQ_API_KEY"))
#llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768",api_key=userdata.get("GROQ_API_KEY"))
#llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0.7),


## crewAI Tools

In [147]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
#  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_guideline = FileReadTool(file_path='./guideline_best_practices_data_analytics.md')
#semantic_search_guideline = MDXSearchTool(mdx='./guideline_best_practices_data_analytics.md')



In [148]:
#from IPython.display import Markdown, display
#display(Markdown("./guideline_best_practices_data_analytics.md"))

# Creating Agents

Agent 1: Sr. Business Analyst for Stock Market


In [149]:
# Agent 1: Stock Market Business Analyst
business_analyst_stock_market = Agent(
    role="Sr. Business Analyst",
    goal="Provide clear and comprehensive Business Requirement Definitions (BRD) "
         "for tech team to implement stock market analysis tools and features.",
#    tools = [data_fetch_tool, analysis_tool, documentation_tool],
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Sr. Business Analyst specializing in stock market analysis, "
        "your expertise lies in translating complex business requirements into "
        "detailed and actionable specifications for development teams. "
        "Your deep understanding of stock market dynamics and analytical tools "
        "ensures that the solutions you help create are both effective and aligned "
        "with business objectives. Your role is pivotal in bridging the gap between "
        "business needs and technical implementation, ensuring seamless and efficient development."
        "You must use yahoofinance as data provider. "
    ),
#    llm = llm,
)


In [150]:
# Agent 2: Senior Python Programmer
senior_python_programmer = Agent(
    role="Senior Python Programmer",
    goal="Implement the Python code based on the Business Requirement Definitions (BRD) "
         "provided by the Sr. Business Analyst for stock market analysis tools and features.",
#   tools = [coding_tool, debugging_tool, testing_tool],
#    tools = [python_repl_tool, read_guideline, semantic_search_guideline],
#    tools = [read_guideline],
    verbose=True,
    backstory=(
        "As a Senior Python Programmer with extensive experience in stock market and data analytics, "
        "you excel at transforming detailed business requirements into robust, efficient, and scalable code. "
        "Your deep understanding of financial data, combined with your coding expertise, "
        "ensures that the solutions you develop are both accurate and performant. "
        "You are adept at collaborating with analysts to fine-tune requirements and deliver high-quality software "
        "that meets business needs and exceeds expectations."
    ),
#    llm = llm
)


In [151]:
# Agent 3: Senior Quality Assurance Analyst
senior_qa_analyst = Agent(
    role="Senior Quality Assurance Analyst",
    goal="Ensure the accuracy, performance, and reliability of the stock market analysis tools "
         "implemented by the Senior Python Programmer, based on the Business Requirement Definitions (BRD).",
#   tools = [testing_tool, validation_tool, reporting_tool],
#    tools = [python_repl_tool, read_guideline, semantic_search_guideline],
#    tools = [read_guideline],
    verbose=True,
    backstory=(
        "As a Senior Quality Assurance Analyst with a specialization in stock market and data analytics, "
        "you bring a wealth of experience in validating and verifying financial software. "
        "Your meticulous approach to testing ensures that all aspects of the implemented solutions are thoroughly evaluated. "
        "You collaborate closely with developers and analysts to identify potential issues, ensuring that the final product "
        "meets the highest standards of quality and performance. Your role is critical in maintaining the integrity and reliability "
        "of stock market analysis tools."
    ),
#    llm = llm
)


## Creating Tasks

In [152]:
# Task for Sr. Business Analyst: Define Business Requirements
business_analyst_stock_market_task = Task(
    description=(
        "Give the {user_story}, analyze the stock market for {ticker_symbol}  "
        "and document detailed Business Requirement Definitions (BRD) for the development team. "
        "Ensure that the requirements cover all aspects of the stock market analysis tools and features, "
        "including data sources, processing, visualization, and reporting."
    ),
    expected_output=(
        "A comprehensive Business Requirement Definition (BRD) document that includes detailed descriptions "
        "of the required stock market analysis tools and features, data sources, processing steps, "
        "visualization needs, and reporting requirements according to the {user_story}."
    ),
    agent=business_analyst_stock_market,
    output_file="business_requirement_data_analysis.md",

#    async_execution=True,
#    llm = llm
)



In [153]:
# Task for Senior Python Programmer: Implement Business Requirements
senior_python_programmer_task = Task(
    description=(
        "Based on the Business Requirement Definitions (BRD) provided by the Sr. Business Analyst and also the {user_story}, "
        "implement the necessary Python code to develop stock market analysis tools and features. "
        "Ensure the code is efficient, scalable, and aligns with the documented requirements. "
        "Collaborate with the QA team to validate and refine the implementation. "
        "You must follow the guideline for Python implementation best practice."
        "You must execute the code to ensure it is working properly."
    ),
    expected_output=(
        "A fully functional and documented set of Python scripts that implement the stock market analysis tools and features "
        "as specified in the BRD. The code should be ready for testing and deployment."
    ),
    agent=senior_python_programmer,
#    async_execution=True,
#    llm = llm,
    output_file="data_analysis_code.md",
    context=[business_analyst_stock_market_task]
)


In [154]:
# Task for Senior Quality Assurance Analyst: Validate Implementation
senior_qa_analyst_task = Task(
    description=(
        "Conduct thorough testing and validation of the stock market analysis tools and features "
        "implemented by the Senior Python Programmer. Ensure that all functionalities align with the Business Requirement Definitions (BRD) "
        " and the {user_story} provided by the Sr. Business Analyst. "
        "Identify and document any issues, and work with the development team to resolve them. "
        "Perform performance testing and ensure the accuracy and reliability of the analysis tools."
        "You must execute the code to ensure it is working properly."
    ),
    expected_output=(
        "A comprehensive test report documenting the results of all tests conducted, "
        "including identified issues, resolutions, and final validation of the stock market analysis tools. "
        "The tools should be confirmed as accurate, reliable, and ready for deployment."
    ),
    agent=senior_qa_analyst,
    context=[business_analyst_stock_market_task,senior_python_programmer_task],
    output_file="data_analysis_qa_document.md",
  #   async_execution=True,
#    llm = llm,
)


- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

## Creating the Crew

In [155]:
data_analysis_stock_market = Crew(
    agents=[business_analyst_stock_market,
            senior_python_programmer,
            senior_qa_analyst],

    tasks=[business_analyst_stock_market_task,
           senior_python_programmer_task,
           senior_qa_analyst_task],
    manager_llm=ChatOpenAI(model="gpt-4-turbo",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [156]:
stock_market_data_analysis_inputs = {
    'user_story': "As a Financial Analyst, I want to evaluate in a visual manner the performance of Citigroup's (C) "
                  "stock over the last 30 days so that I can make informed decisions about potential "
                  "investment opportunities and provide accurate recommendations to stakeholders."
                  "I do need to see the evaluation result in a visual manner.",
    'ticker_symbol': 'C'
}



In [157]:
### this execution will take a few minutes to run
result = data_analysis_stock_market.kickoff(inputs=stock_market_data_analysis_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Give the As a Financial Analyst, I want to evaluate in a visual manner the performance of Citigroup's (C) stock over the last 30 days so that I can make informed decisions about potential investment opportunities and provide accurate recommendations to stakeholders.I do need to see the evaluation result in a visual manner., analyze the stock market for C  and document detailed Business Requirement Definitions (BRD) for the development team. Ensure that the requirements cover all aspects of the stock market analysis tools and features, including data sources, processing, visualization, and reporting.


> Entering new CrewAgentExecutor chain...
Thought: To create a comprehensive Business Requirement Definitions (BRD) for evaluating the performance of Citigroup's stock over the last 30 days visually, I first need to understand the specific features and functionalities the financial analyst requires for the stock market analysis

In [158]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

./tailored_resume.md

In [159]:
display(Markdown("./interview_materials.md"))

./interview_materials.md